In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-large-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-large-msmarco')
model = model.to(device)


In [3]:
def get_qg(doc_text):

    input_ids = tokenizer.encode(doc_text, truncation=True, max_length=512, padding="longest", return_tensors='pt').to(device)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_k=10,
        num_return_sequences=50)

    return list(set([tokenizer.decode(outputs[i], skip_special_tokens=True) for i in range(50)]))

len(get_qg("Hello world is a rudementary check for every programming language!"))

18

In [9]:
import csv
import sys
from tqdm import tqdm

csv.field_size_limit(sys.maxsize)

with open("../data/fulldocs.tsv", "rt") as f_reader:
    tsv_reader = csv.reader(f_reader, delimiter="\t")
    with open('../data/qg.txt', 'w') as f:
        for row in tqdm(tsv_reader):
            qg_list = get_qg(row[-1])
            
            for question in qg_list:
                f.write(question)
                f.write('\n')

40225it [19:04:09,  1.71s/it]


KeyboardInterrupt: 

In [11]:
with open('../data/qg.txt', 'r') as f:
    lines = f.readline()

In [12]:
lines

'what is the energy of the rigel\n'